# PDF Merger

## Step 1 break all pdf in individual files and assign proper file names to recogise

In [6]:
import glob

from PyPDF2 import PdfWriter, PdfReader
import pdfquery

from pypdf import PdfWriter
import glob
import os

In [4]:
stage1_input = './data/*.pdf'

for filename in glob.glob(stage1_input):
    input_pdf = PdfReader(open(filename, "rb"))

    for i in range(len(input_pdf.pages)):
        qpdf = pdfquery.PDFQuery(filename)
        qpdf.load(i)
        id = qpdf.pq(
            'LTTextLineHorizontal:in_bbox("137.93, 137.518, 186.399, 147.478")').text()
        # 137.93, 137.518, 186.399, 147.478
        res = id.strip().strip('-').strip()

        output = PdfWriter()
        output.add_page(input_pdf.pages[i])
        res = res.replace("/", "_")
        with open("./stage1_out/%s.pdf" % res, "wb") as outputStream:
            output.write(outputStream)

## Stage 2 combine pdf files based on similar names

In [8]:
globnames = glob.glob("./stage1_out/*.pdf")

finalnames = []
for filepath in globnames:
    filename = os.path.basename(filepath)  # Gets "1001 - 2_2.pdf"
    filename = os.path.splitext(filename)[0]  # Removes .pdf extension
    finalnames.append(filename)

merger = PdfWriter()
prev = -1

for filename, file in zip(finalnames, globnames):
    filename = str(filename)
    filename = filename.strip()
    filename = filename.split(" - ")[0].strip()
    if prev == filename or prev == -1:
        merger.append(file)
        prev = filename

    else:
        merger.write(f"./stage2_out/{prev}.pdf")
        merger.close()
        merger = PdfWriter()
        merger.append(file)
        prev = filename

merger.write(f"./stage2_out/{prev}.pdf")
merger.close()